In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import ITunesAmazonStructuredBenchmark

benchmark = ITunesAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

17:39:24 INFO:Extracting iTunes-Amazon-Structured...
17:39:24 INFO:Reading iTunes-Amazon-Structured row_dict...
17:39:24 INFO:Reading iTunes-Amazon-Structured train.csv...
17:39:24 INFO:Reading iTunes-Amazon-Structured valid.csv...
17:39:24 INFO:Reading iTunes-Amazon-Structured test.csv...


<ITunesAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/iTunes-Amazon/itunes_amazon_exp_data.zip

## Preprocess

In [5]:
attr_list = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/62830 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_config_dict = {
    'Song_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Song_Name': {
        'source_attr': 'Song_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'Artist_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Artist_Name': {
        'source_attr': 'Artist_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'Album_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Album_Name': {
        'source_attr': 'Album_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'Genre': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_Genre': {
        'source_attr': 'Album_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'Price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'CopyRight': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'Time': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'Released': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

17:39:25 INFO:For attr=Song_Name, computing actual max_str_len
17:39:25 INFO:actual_max_str_len=21 must be even to enable NN pooling. Updating to 22
17:39:25 INFO:For attr=Song_Name, using actual_max_str_len=22
17:39:25 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:29 INFO:For attr=Artist_Name, computing actual max_str_len
17:39:29 INFO:For attr=Artist_Name, using actual_max_str_len=20
17:39:29 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:33 INFO:For attr=Album_Name, computing actual max_str_len
17:39:33 INFO:For attr=Album_Name, using actual_max_str_len=16
17:39:33 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:37 INFO:For attr=Genre, computing actual max_str_len
17:39:37 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
17:39:37 INFO:For attr=Genre, using actual_max_str_len=14
17:39:37 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:40 INFO:For attr=Price, computing actual max_str_len
17:39:40 INFO:F

{'Song_Name': AttrConfig(source_attr='Song_Name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_Song_Name': AttrConfig(source_attr='Song_Name', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', '

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=128,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    row_numericalizer=row_numericalizer,
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStoppingMinEpochs(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'matcher-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
17:39:41 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:39:41 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

17:39:41 INFO:Train pair count: 78
17:39:41 INFO:Valid pair count: 27
17:39:41 INFO:Test pair count: 27
17:39:41 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 26.2 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
16.1 M    Trainable params
10.1 M    Non-trainable params
26.2 M    Total params
104.738   Total estimated model params size (MB)
17:41:09 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 26.2 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
16.1 M    Trainable params
10.1 M    Non-trainable params
26.2 M    Total params
104.738   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [15]:
model.blocker_net.get_signature_weights()

{'Song_Name': 0.10422512888908386,
 'semantic_Song_Name': 0.10374309867620468,
 'Artist_Name': 0.07764952629804611,
 'semantic_Artist_Name': 0.08895979076623917,
 'Album_Name': 0.08335656672716141,
 'semantic_Album_Name': 0.09908919781446457,
 'Genre': 0.06361004710197449,
 'semantic_Genre': 0.09879519790410995,
 'Price': 0.06450369209051132,
 'CopyRight': 0.06946979463100433,
 'Time': 0.08490937948226929,
 'Released': 0.06168855354189873}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

17:42:43 INFO:Validating best model...


{'valid_f1_at_0.3': 0.7941176470588235,
 'valid_f1_at_0.5': 0.8437499999999999,
 'valid_f1_at_0.7': 0.9,
 'valid_f1_at_0.9': 0.9259259259259259,
 'valid_pair_entity_ratio_at_0.3': 0.7735849056603774,
 'valid_pair_entity_ratio_at_0.5': 0.6981132075471698,
 'valid_pair_entity_ratio_at_0.7': 0.6226415094339622,
 'valid_pair_entity_ratio_at_0.9': 0.5094339622641509,
 'valid_precision_at_0.3': 0.6585365853658537,
 'valid_precision_at_0.5': 0.7297297297297297,
 'valid_precision_at_0.7': 0.8181818181818182,
 'valid_precision_at_0.9': 0.9259259259259259,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 1.0,
 'valid_recall_at_0.7': 1.0,
 'valid_recall_at_0.9': 0.9259259259259259}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

17:42:45 INFO:Train pair count: 78
17:42:45 INFO:Valid pair count: 27
17:42:45 INFO:Test pair count: 27
17:42:45 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.7941176470588235,
  'test_f1_at_0.5': 0.9310344827586207,
  'test_f1_at_0.7': 0.9473684210526316,
  'test_f1_at_0.9': 0.9259259259259259,
  'test_pair_entity_ratio_at_0.3': 0.7592592592592593,
  'test_pair_entity_ratio_at_0.5': 0.5740740740740741,
  'test_pair_entity_ratio_at_0.7': 0.5555555555555556,
  'test_pair_entity_ratio_at_0.9': 0.5,
  'test_precision_at_0.3': 0.6585365853658537,
  'test_precision_at_0.5': 0.8709677419354839,
  'test_precision_at_0.7': 0.9,
  'test_precision_at_0.9': 0.9259259259259259,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 1.0,
  'test_recall_at_0.7': 1.0,
  'test_recall_at_0.9': 0.9259259259259259}]